# Mobile App for Lottery Addiction

医療機関が宝くじ中毒者むけに[6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49)で当選する確率を見積もれるアプリの作成を依頼してきた、と仮定する。本プロジェクトの目的はこのアプリのコアとなる、当選確率の計算を行う関数の作成である。

# Core Functions

階乗と組み合わせを計算する関数を作る

In [1]:
def factrial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def combinations(n, k):
    numerator = factrial(n)
    denominator = factrial(k) * factrial(n-k)
    return numerator / denominator

# One-ticket Probability

宝くじを一枚買った時に、一等が当たる確率を計算する機能を搭載する。ロト6/49の宝くじには、1から49の49個の数字のうち6つの数字が数列の形で記されている。一等に当選するためには、自分の持っている宝くじの数列が本数字の数列と完全に一致する必要がある。

はじめに述べた機能を作るにあたり、以下のことに注意する：
* アプリ内で、ユーザーは1から49のうち6つの異なる数字を入力する。
* 6つの数字はPythonのリストとして扱い、すなわち、関数への入力はひとつである。
* 確率の知識がないユーザーでも理解しやすいように出力方法を工夫する。

In [2]:
def one_ticket_probability(user_numbers):
    n_combinations = combinations(49, 6)
    probability_one_ticket = 1/n_combinations
    percentage_form = probability_one_ticket * 100
    
    print('''あなたの宝くじの数列が{}の時、一等が当たる確率は{:.7f}％です。つまり、一等が当たる可能性は{:,}分の1です。'''
          .format(user_numbers, percentage_form, int(n_combinations)))

関数が機能するかどうか異なる２つのリストを用いてテストする。ここで、関数が正しければ、テストで出力される確率は一致する。

In [3]:
list_1 = [1, 33, 45, 25, 6, 8]
one_ticket_probability(list_1)

あなたの宝くじの数列が[1, 33, 45, 25, 6, 8]の時、一等が当たる確率は0.0000072％です。つまり、一等が当たる可能性は13,983,816分の1です。


In [4]:
list_2 = [3, 21, 22, 43, 5, 34]
one_ticket_probability(list_2)

あなたの宝くじの数列が[3, 21, 22, 43, 5, 34]の時、一等が当たる確率は0.0000072％です。つまり、一等が当たる可能性は13,983,816分の1です。


# Historical Data Check for Canada Lottery

ユーザーが自分のロトチケットと過去のロトデータとを見比べて、今までに勝っていたかどうか確かめることができるようにする。
まずは、ロトデータ`649.csv`について知ることから始める。

In [5]:
import pandas as pd

lottery_canada = pd.read_csv('649.csv')
print(lottery_canada.shape)

(3665, 11)


In [6]:
print(lottery_canada.head(3))
print(lottery_canada.tail(3))

   PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
0      649            1                0  6/12/1982               3   
1      649            2                0  6/19/1982               8   
2      649            3                0  6/26/1982               1   

   NUMBER DRAWN 2  NUMBER DRAWN 3  NUMBER DRAWN 4  NUMBER DRAWN 5  \
0              11              12              14              41   
1              33              36              37              39   
2               6              23              24              27   

   NUMBER DRAWN 6  BONUS NUMBER  
0              43            13  
1              41             9  
2              39            34  
      PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
3662      649         3589                0  6/13/2018               6   
3663      649         3590                0  6/16/2018               2   
3664      649         3591                0  6/20/2018              14   

     

# Function for Historical Data Check

ユーザーが自分のチケットと過去の本数字を比較して、これまでにチケットの数列が一等と一致していたことがあるかどうかを確かめることができるような機能を作る。

以下の点に注意する：
* ユーザーは1〜49のうち6つの異なる数字を入力する。
* その数列を、Pythonのリストとして扱い、関数にリストを入力する。
* チケットの数列が過去の本数字と一致した回数と、次の抽選で一等をとる確率を、わかりやすく出力する。

In [7]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winning_numbers = lottery_canada.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [8]:
def check_historical_occurrence(user_numbers, historical_numbers):   
    '''
    user_numbers: a Python list
    historical numbers: a pandas Series
    '''
    
    user_numbers_set = set(user_numbers)
    check_occurrence = historical_numbers == user_numbers_set
    number_matchs = check_occurrence.sum()
    
    if number_matchs == 0:
        print('''{}は過去に一度も本数字になったことがありません。\n一等が当たる確率は0.0000072％です。つまり、一等が当選する可能性は13,983,816分の1です'''
             .format(user_numbers))
    else:
        print('''{}は過去に{}回本数字になったことがあります。\n一等が当たる確率は0.0000072％です。つまり、一等が当選する可能性は13,983,816分の1です'''
             .format(user_numbers, number_matchs))  

テストする。

In [9]:
test_1 = [1, 33, 45, 25, 6, 8]
check_historical_occurrence(test_1, winning_numbers)

[1, 33, 45, 25, 6, 8]は過去に一度も本数字になったことがありません。
一等が当たる確率は0.0000072％です。つまり、一等が当選する可能性は13,983,816分の1です


In [10]:
test_2 = [3, 41, 11, 12, 43, 14]
check_historical_occurrence(test_2, winning_numbers)

[3, 41, 11, 12, 43, 14]は過去に1回本数字になったことがあります。
一等が当たる確率は0.0000072％です。つまり、一等が当選する可能性は13,983,816分の1です


 # Multi-ticket Probability
 
 1枚以上(1~13983826)チケットを購入したときに、一等を当選する確率を出力する関数を作る

In [11]:
def multi_ticket_probability(n_tickets):
    n_combination = combinations(49, 6)
    probability = n_tickets / n_combination
    percentage = probability * 100
    
    if n_tickets == 1:
        print('''チケット一枚で一等が当たる確率は{:.7f}％です。この時、一等が当選する可能性は{:,}分の1です。'''
             .format(percentage, int(n_combination)))
    else:
        combination_simplified = round(n_combination / n_tickets)
        print('''チケット{}種類で一等が当たる確率は{:.7f}％です。この時、一等が当選する可能性は{:,}分の1です。'''
             .format(n_tickets, percentage, combination_simplified))

テスト

In [12]:
test_numbers = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for number in test_numbers:
    multi_ticket_probability(number)
    print('------------------------')

チケット一枚で一等が当たる確率は0.0000072％です。この時、一等が当選する可能性は13,983,816分の1です。
------------------------
チケット10種類で一等が当たる確率は0.0000715％です。この時、一等が当選する可能性は1,398,382分の1です。
------------------------
チケット100種類で一等が当たる確率は0.0007151％です。この時、一等が当選する可能性は139,838分の1です。
------------------------
チケット10000種類で一等が当たる確率は0.0715112％です。この時、一等が当選する可能性は1,398分の1です。
------------------------
チケット1000000種類で一等が当たる確率は7.1511238％です。この時、一等が当選する可能性は14分の1です。
------------------------
チケット6991908種類で一等が当たる確率は50.0000000％です。この時、一等が当選する可能性は2分の1です。
------------------------
チケット13983816種類で一等が当たる確率は100.0000000％です。この時、一等が当選する可能性は1分の1です。
------------------------


# Less Winning Numbers

6つの数字のうち、2~5つが一致する確率を求める関数を定義する

In [13]:
def probability_less_6(n_winning_numbers):
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)
    prize_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_combination = combinations(49, 6)
    probability = prize_outcomes / n_combination
    
    percentage = probability * 100
    combination_simplified = round(n_combination / prize_outcomes)
    print('''数字が{}つ一致する確率は{:.6f}％です。すなわち、{}分の1です。'''
         .format(n_winning_numbers, percentage, int(combination_simplified)))

In [14]:
test_2_5 = [2, 3, 4, 5]

for i in test_2_5:
    print(probability_less_6(i))

数字が2つ一致する確率は13.237803％です。すなわち、8分の1です。
None
数字が3つ一致する確率は1.765040％です。すなわち、57分の1です。
None
数字が4つ一致する確率は0.096862％です。すなわち、1032分の1です。
None
数字が5つ一致する確率は0.001845％です。すなわち、54201分の1です。
None
